## Object Detection

Hoping at this point, we are familiar with classification, object detection can be explained as a classification with localization.

### Classification

![mater](assets/mcqueen_real.png)

### with Localization (and over multiple objects)

![mcqueen](assets/mcqueen.jpeg)

## Where to use?

- Could be used on any kind of task where finding the location of the object(s) are of any use
- Anything related to traffic, pedestrians, types of vehicles, drivable roads, landing zones etc.
- Anything related to locating a disease over some type of medical imaging (MRI, Ultrasound, CT ...)
- When designing automated stores, factories etc. (Like Amazon Go cashierless stores)

this could go on

## Yea yea yea its all good but how does it come to be and how can I learn / use it?

Okay we are kind of familiar with a CNN, it acts as a feature extractor, connects to a FCN with number of classes as neurons for output and ta-dah, we have a multi class classifier. 

![out_neurons](assets/detection_output_neurons.png)

![annotated_out_neurons](assets/annotated_output_neurons.png)

In [8]:
# This can be penalized with any loss but main logic here is that 
loss_fn = lambda x, y: (x - y) ** 2
prediction = [1] * 8  # P, x, y, w, h, c1, c2, c3
label = [1] * 8

if prediction[0]:
    # calculate loss for only first neuron, we want it to be 0
    loss = loss_fn(prediction[0], label[0])
else:
    # calculate loss over all the other predictions as well
    loss = sum([loss_fn(p, l) for p, l in zip(prediction, label)])  # you do not have to use one type of loss function here
    # you can use variation of losses which may differ from a bounding box to a class

## But how do we classify an unknown number of objects?

### Let me explain while expanding on some utility functions that make object detection the way it is

#### Sliding window detection

**Sliding Window Detection: Because who needs an exact address when you can just keep 
looking?**

Sliding window detection is like searching for your car in a crowded parking lot - 
except instead of cars, it's objects in images! This method involves repeatedly 
applying the same feature detector (or "window") to an image at multiple locations 
and scales. As it slides around, it checks each spot to see if there's a good match.

**Example:** Imagine you're looking for your cat hiding behind a couch in a large 
living room. A sliding window detector would move its feature template over the 
image, checking every possible location where your cat might be hiding, at different 
scales and orientations. Eventually, it'll find that sneaky kitty!


![](assets/sliding_snail.gif)

In [ ]:
from typing import Tuple, Iterable
import numpy as np

def sliding_window(image: np.array, step_size: int, window_size: Tuple[int, int]) -> Iterable[int, int, np.ndarray]:
    H, W = image.shape  # considering image is 2 channels, you should put a check here
    for y in range(0, H, step_size):
        for x in range(0, W, step_size):
            yield (x, y, image[y: y + window_size[1], x: x + window_size[0]])

#### Intersection Over Union (IoU)

**IoU: Because parking your spaceship in the correct spot is crucial**

IoU, or Intersection over Union, is a measure of how well two things fit together... 
literally! It's often used in object detection and computer vision tasks, like 
identifying cats in images (which can be quite challenging, let's be honest). IoU 
simply calculates the ratio of the intersection area to the union area between two 
bounding boxes.

**Example:** Imagine you're playing a game where you need to draw a box around a cat 
in an image. If your box perfectly overlaps with the actual cat, that's a high IoU 
score (ideally 1). But if your box is way off, only partially covering the cat... not 
so much.

![](assets/iou-formula.webp)

![](assets/iou-example.png)

[image_1 reference](https://idiotdeveloper.com/what-is-intersection-over-union-iou/)

[image_2 reference](https://www.superannotate.com/blog/intersection-over-union-for-object-detection)

In [ ]:
def iou(pred: Tuple[int, int, int, int], 
        gt: Tuple[int, int, int, int]) -> float:
    """ in xyxy format, you can write it as xywh format if you'd like """
    # intersection points
    x1 = max(pred[0], gt[0])
    y1 = max(pred[0], gt[0])
    x2 = max(pred[0], gt[0])
    y2 = max(pred[0], gt[0])

    # intersection
    intersection = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)

    # area of boxes
    area_pred = (pred[2] - pred[0] + 1) * (pred[3] - pred[1] + 1)
    area_gt = (gt[2] - gt[0] + 1) * (gt[3] - gt[1] + 1)

    iou = intersection / float(area_pred + area_gt - intersection)
    return iou

#### Non-Max Suppression (NMS)

**NMS: Because you can't have multiple cats in one box (or can you?)**

NMS, or Non-Maximum Suppression, is like the ultimate party crasher... for bounding 
boxes! When an object detector spits out multiple bounding boxes around the same 
thing (like that pesky cat), NMS comes to the rescue. It looks at these boxes and 
says: "No way, buddy! There can only be one correct box here." Then it selectively 
suppresses or removes redundant boxes, leaving you with a more accurate and tidier 
set of predictions.

**Example:** Imagine you're playing a game where you need to detect all the cats in 
an image. If your detector incorrectly identifies multiple cat boxes around the same 
furry friend, NMS will help eliminate those duplicates, giving you a cleaner and more 
precise detection result!


![](assets/nms.png)

[image reference](https://learnopencv.com/weighted-boxes-fusion/)

In [ ]:
def non_max_supression(boxes: list, scores: list, threshold: float = 0.5):
    if len(boxes) == 0:
        return []  # no prediction to supress
    
    # it is good to work with np arrays / easier if it is not already that way
    boxes = np.array(boxes)
    scores = np.array(scores)

    picked = []
    # sorting bbox confidence scores in descending order
    indices = np.argsort(scores)[::-1]

    while len(indices) > 0:
        current = indices[0]
        picked.append(current)

        # compute iou for all of the rest
        remaining = indices[1:]
        ious = np.array([iou(boxes[current], boxes[i]) for i in remaining])

        indices = remaining[ious < threshold]  # elliminate boxes that computes iou less than the threshold

    return boxes[picked]

#### Anchor Boxes

**Anchor Boxes: Because who needs precision when you can have options?**

Anchor boxes are like the buffet of object detection - they offer multiple choices (or "anchors") for bounding box predictions. Instead of predicting a single box, an anchor box-based detector proposes a range of possible boxes that might contain an object. This is like saying: "Hey, I'm not sure if this here or there... but it's probably around here somewhere!"

**Example:** Imagine you're trying to detect all the animals in an image. An anchor box-based detector would propose multiple bounding boxes with different sizes and aspect ratios, covering possible locations and orientations of the animals. The algorithm then adjusts these anchors based on the detected objects' characteristics, like size and shape, to get a more accurate detection result!


In [ ]:
def generate_anchor_boxes(scales: list, aspect_ratios: list, image_size: Tuple[int, int]):
    anchor_boxes = []
    for scale in scales:
        for ratio in aspect_ratios:
            width = scale * np.sqrt(ratio)
            height = scale / np.sqrt(ratio)
            # Create anchor box
            anchor_boxes.append([width, height])
    return anchor_boxes

#### Deciding bbox locations based on anchor boxes

In [ ]:
sigmoid = lambda x: (1 / (1 + np.exp(-x)))

def decode_bbox(predicted_bbox, anchor_bbox, grid_cell, stride):
    # converting to absolute coordinates w.r.t. anchor box
    bx = (sigmoid(predicted_bbox[0]) + grid_cell[0]) * stride
    by = (sigmoid(predicted_bbox[1]) + grid_cell[1]) * stride
    bw = anchor_bbox[0] * np.exp(predicted_bbox[2])
    bh = anchor_bbox[1] * np.exp(predicted_bbox[3])
    
    return [bx, by, bw, bh]

#### Side note on how YOLO calculates loss

(maybe not the current ones like YOLO7-8-9-10..., can't keep track of them)

In [ ]:
iou_loss_fn, bce, categorical_ce = None, None, None

def yolo_loss(predictions, ground_truth, anchors):
    # Split predictions into components
    obj_preds = predictions[..., 0]   # objectness
    box_preds = predictions[..., 1:5]  # x, y, w, h
    class_preds = predictions[..., 6:] # class predictions
    
    # "is there" an object?
    obj_loss = bce(obj_preds, ground_truth[..., 0])
    
    # "how much" of the object we have correctly guessed
    iou_loss = iou_loss_fn(box_preds, ground_truth[..., 1:5])
    
    # did we guess "which" object it is
    class_loss = categorical_ce(class_preds, ground_truth[..., 6:])
    
    return iou_loss + obj_loss + class_loss

In [ ]:
# pred stuff on yolo